In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_excel('C:/Users/ScMa9004/Documents/RI/RIEXPORT_31.01 (1).xlsx')

In [4]:
mp = pd.read_csv('C:/Users/ScMa9004/Documents/RI/AREA_CELL_MAPPING_1580455751443.csv')

In [5]:
arc = pd.read_excel('C:/Users/ScMa9004/Documents/All Russia cities 01.01.2019_ for_2020.xlsx',
                    sheet_name = 'All_Russia_cities', 
                    skiprows = 1)

In [6]:
uo200 = pd.read_excel('C:/Users/ScMa9004/Documents/Universe Over 200.xls', sheet_name = 'Towns&Villages', skiprows = 1)
uo200 = uo200[uo200['ACN NN'].isna() | (uo200['ACN NN'] > 9999)]

In [7]:
ops = pd.read_excel('C:/Users/ScMa9004/Documents/Справочник OPS_регион_QCT_2.2020.xlsx', sheet_name = 'данные')

In [8]:
rules = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/FMCG_CIG_BEER.xlsx',
                      sheet_name = 'index + mcellname')

In [9]:
def parser(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU', 'OLA', 'OME', 'OSM', 'OPV', 'OMM', 'OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT|TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
                types = ['OKS', 'ONS', 'OPV', 'OSM']
                hip = [mcell.split('-')[2]]
            else:
                types = list(mcell.split('-')[2].split('|'))
                hip = []
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        hip = []
        if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
            types = ['OKS', 'ONS', 'OPV', 'OSM']
            hip = [mcell.split('-')[2]]
        else:
            types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

In [10]:
def free_search(df, mp):
    
    kf = df[df['Stage'] == 'FRAME EXHAUST'].reset_index(drop = True)
    
    typ = pd.pivot_table(kf[['Sample Area ID', 'DA_STORE_TYPE']], values = 'DA_STORE_TYPE', index = 'Sample Area ID', 
              aggfunc = lambda x: list(x.unique())).reset_index()
    
    pt = pd.DataFrame()
    
    pt = pd.pivot_table(kf, values='BatchID', index=['SampleAreaName', 'Sample Area ID'], columns=['Index Name'],
                        aggfunc = lambda x: len(x.unique()))
    pt = pt.reset_index()
    
    pt['MICRO_CELL_RULE_ID'] = np.nan
    
    pt = pt.rename(columns={"RU BEER AUDIT - DUPLICATE": "BEER",
                            "RU CIG AUDIT - DUPLICATE": "CIG",
                            "RU FMCG AUDIT - DUPLICATE": "FMCG",
                            "RU EXXON AUDIT": "EXXON"
                            })
    
    for i in range(len(pt['Sample Area ID'])):
        try:
            pt.loc[i, 'MICRO_CELL_RULE_ID'] = mp['MICRO_CELL_RULE_ID'][pt['Sample Area ID'][i] == mp['SAMPLE_AREA_ID']].iloc[0]
        except:
            try:
                pt.loc[i, 'MICRO_CELL_RULE_ID'] = mp['MICRO_CELL_RULE_ID'][(pt['Sample Area ID'][i] == mp['SAMPLE_AREA_ID']) &
                                                                       (mp['INDEX_ID'] == 1000334)].iloc[0]
            except:
                pt.loc[i, 'MICRO_CELL_RULE_ID'] = mp['MICRO_CELL_RULE_ID'][(pt['Sample Area ID'][i] == mp['SAMPLE_AREA_ID']) & 
                                                                       (mp['INDEX_ID'] == 1000296)].iloc[0]
          
    try:
        pt['EXXON']
    except:
        pt['EXXON'] = np.nan
        
    pt[['FMCG','BEER', 'CIG', 'EXXON']] =  pt[['FMCG','BEER', 'CIG', 'EXXON']].fillna(0)

    for i in pt.index:
        if max(pt['FMCG'][i], pt['BEER'][i], pt['CIG'][i], pt['EXXON'][i]) > 1:
            for j in range(int(max(pt['FMCG'][i], pt['BEER'][i], pt['CIG'][i], pt['EXXON'][i])) - 1):
                pt = pt.append(pt.iloc[i])
                
    pt[['FMCG','BEER', 'CIG', 'EXXON']] = pt[['FMCG','BEER', 'CIG', 'EXXON']].replace(0, np.nan)
    
    pt = pt.sort_index()
    
    k = -1
    for i in range(len(pt.index)):
        if pt.index[i] == k:
            pt['FMCG'].iloc[i] = pt['FMCG'].iloc[i - 1] - 1
            pt['CIG'].iloc[i] = pt['CIG'].iloc[i - 1] - 1 
            pt['BEER'].iloc[i] = pt['BEER'].iloc[i - 1] - 1
            pt['EXXON'].iloc[i] = pt['EXXON'].iloc[i - 1] - 1
        else:
             k = pt.index[i] 
    
    pt.loc[pt['FMCG'] < 1, 'FMCG'] = np.nan
    pt.loc[pt['BEER'] < 1, 'BEER'] = np.nan
    pt.loc[pt['CIG'] < 1, 'CIG'] = np.nan
    pt.loc[pt['EXXON'] < 1, 'EXXON'] = np.nan
    
    pt.loc[pt['FMCG'].notna(), 'FMCG'] = 1
    pt.loc[pt['BEER'].notna(), 'BEER'] = 1
    pt.loc[pt['CIG'].notna(), 'CIG'] = 1
    pt.loc[pt['EXXON'].notna(), 'EXXON'] = 1
    
    pt = pt.reset_index(drop = True)
    
    for i in range(len(pt['Sample Area ID'])):
        if pt['FMCG'][i] == 1:
            pt.loc[i, 'FMCG'] = kf['BatchID'][pt['Sample Area ID'][i] == kf['Sample Area ID']].iloc[0]
            kf = kf.drop(kf[pt['Sample Area ID'][i] == kf['Sample Area ID']].index[0])
        if pt['BEER'][i] == 1:
            pt.loc[i, 'BEER'] = kf['BatchID'][pt['Sample Area ID'][i] == kf['Sample Area ID']].iloc[0]
            kf = kf.drop(kf[pt['Sample Area ID'][i] == kf['Sample Area ID']].index[0])
        if pt['CIG'][i] == 1:
            pt.loc[i, 'CIG'] = kf['BatchID'][pt['Sample Area ID'][i] == kf['Sample Area ID']].iloc[0]
            kf = kf.drop(kf[pt['Sample Area ID'][i] == kf['Sample Area ID']].index[0])
        if pt['EXXON'][i] == 1:
            pt.loc[i, 'EXXON'] = kf['BatchID'][pt['Sample Area ID'][i] == kf['Sample Area ID']].iloc[0]
            kf = kf.drop(kf[pt['Sample Area ID'][i] == kf['Sample Area ID']].index[0])
    
    pt['cluster'] = pt['SampleAreaName'].str.split(n = 1, expand = True)[0]
        
    pt = pt[['SampleAreaName', 'Sample Area ID', 'FMCG', 'BEER', 'CIG', 'EXXON', 'MICRO_CELL_RULE_ID', 'cluster']]
    
#    typ = pd.pivot_table(kf[['Sample Area ID', 'DA_STORE_TYPE']], values = 'DA_STORE_TYPE', index = 'Sample Area ID', 
#              aggfunc = lambda x: list(x.unique())).reset_index()
    
#    pt = pd.merge(pt, typ)
    
    return pt

In [11]:
pt = free_search(df, mp)

C:\Users\ScMa9004\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
pt['rural'] = np.nan
for i, j in pt.iterrows():
    if 'RR' in j['SampleAreaName']:
        pt.loc[i, 'rural'] = 1 

In [13]:
#fmcg
fmcg = pt[pt['FMCG'].notna()].reset_index(drop = True)

fmcg_fs = pd.DataFrame(columns = ['Sample Area ID', 'cluster', 'ccode', 'cname', 'zadanie'])

n = 0
for i, j in fmcg.iterrows():
    data = parser(rules['new_mcell_name'][rules['MCELL_ID'] == j['MICRO_CELL_RULE_ID']].iloc[0])
    if len(data[1]) == 1:
        fmcg_fs.loc[n, 'cname'] =  data[0][0]
        fmcg_fs.loc[n, 'ccode'] =  data[1][0]
        fmcg_fs.loc[n, 'zadanie'] = j['Sample Area ID']
        fmcg_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
        fmcg_fs.loc[n, 'cluster'] = j['cluster']
        n += 1
    elif len(data[1]) == 0:
        cities = arc[['City Code', 'City Name (english)']][arc['# of Federal Subject'].isin(data[2]) &
                                                           arc['pgr1'].isin(data[4]) & 
                                                           (arc['Restrict for audit '] == 0) & 
                                                           (arc['RU_CCN '] == 'OTHER') & 
                                                        (arc['RMS_CLUSTER_CODE (справочник)'].astype('str') == j['cluster'])]
        for k in cities.index:
            fmcg_fs.loc[n, 'cname'] =  cities['City Name (english)'][k]
            fmcg_fs.loc[n, 'ccode'] =  cities['City Code'][k]
            fmcg_fs.loc[n, 'zadanie'] = 'OTHER&' + str(int(j['Sample Area ID']))
            fmcg_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
            fmcg_fs.loc[n, 'cluster'] = j['cluster']
            n += 1
            
fmcg_fs = pd.merge(fmcg_fs, ops[['ccode', 'region', 'qct_name_NEW']].drop_duplicates(), on= 'ccode', how = 'left')
#fmcg_fs.to_excel('fmcg_free_search.xlsx')      

In [36]:
#cig 
cig = pt[pt['CIG'].notna() & pt['FMCG'].isna()].reset_index(drop = True)

cig_fs = pd.DataFrame(columns = ['Sample Area ID', 'cluster', 'ccode', 'cname', 'zadanie',
                                 'pgr', 'Distance Strata', 'sub id', 'sub name', 'rayon'])

n = 0
for i, j in cig.iterrows():
    data = parser(rules['new_mcell_name'][rules['MCELL_ID'] == j['MICRO_CELL_RULE_ID']].iloc[0])
    if len(data[1]) == 1:
        cig_fs.loc[n, 'cname'] =  data[0][0]
        cig_fs.loc[n, 'ccode'] =  data[1][0]
        cig_fs.loc[n, 'zadanie'] = j['Sample Area ID']
        cig_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
        cig_fs.loc[n, 'cluster'] = j['cluster']
        n += 1
    elif len(data[1]) == 0:
        if j['rural'] == 1:
            cities = uo200[uo200['Federal Subject Number'].isin(data[2]) & 
                                                uo200['pgr_MT'].isin(data[4]) & 
                                                uo200['Distance Strata'].isin(data[8]) & 
                                                (uo200['RMS_CLUSTER_CODE (ACR)'].astype('str') == j['cluster'])]
            for k in cities.index:
                cig_fs.loc[n, 'cname'] =  cities['Town  / Village Name'][k]
                cig_fs.loc[n, 'ccode'] =  cities['ACN NN'][k]
                cig_fs.loc[n, 'zadanie'] = 'RURAL&' + str(int(j['Sample Area ID'])) 
                cig_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
                cig_fs.loc[n, 'cluster'] = j['cluster']
                cig_fs.loc[n, 'pgr'] = cities['pgr_MT'][k]
                cig_fs.loc[n, 'Distance Strata'] = cities['Distance Strata'][k]
                cig_fs.loc[n, 'sub id'] = cities['Federal Subject Number'][k]
                cig_fs.loc[n, 'sub name'] = cities['Federal Subject Name (Eng)'][k]
                cig_fs.loc[n, 'rayon'] = cities['Rayon (for Villages only)'][k]
                n += 1
        else:
            cities = arc[['City Code', 'City Name (english)']][arc['# of Federal Subject'].isin(data[2]) & 
                                                               arc['pgr1'].isin(data[4]) & 
                                                               (arc['Restrict for audit '] == 0) & 
                                                               (arc['City Code'][~arc['City Code'].isin(data[6])]) & 
                                                        (arc['RMS_CLUSTER_CODE (справочник)'].astype('str') == j['cluster'])]
            for k in cities.index:
                cig_fs.loc[n, 'cname'] =  cities['City Name (english)'][k]
                cig_fs.loc[n, 'ccode'] =  cities['City Code'][k]
                cig_fs.loc[n, 'zadanie'] = 'OTHER&' + str(int(j['Sample Area ID'])) 
                cig_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
                cig_fs.loc[n, 'cluster'] = j['cluster']
                n += 1
    #microcells with 2 and more cities (# 5|7|21|22-STH-OPS-TT-CIG)
    else:
        cities = arc[['City Code', 'City Name (english)']][(arc['Restrict for audit '] == 0) & 
                                                           (arc['City Code'][arc['City Code'].isin(data[1])]) & 
                                                        (arc['RMS_CLUSTER_CODE (справочник)'].astype('str') == j['cluster'])]
        for k in cities.index:
            cig_fs.loc[n, 'cname'] =  cities['City Name (english)'][k]
            cig_fs.loc[n, 'ccode'] =  cities['City Code'][k]
            cig_fs.loc[n, 'zadanie'] = 'OTHER&' + str(int(j['Sample Area ID'])) 
            cig_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
            cig_fs.loc[n, 'cluster'] = j['cluster']
            n += 1
                
rural_fs = cig_fs[cig_fs['ccode'].isna() | (cig_fs['ccode'] > 9999)].reset_index(drop = True)
cig_fs = cig_fs[~cig_fs['ccode'].isna() & (cig_fs['ccode'] < 10000)].drop(columns = {'pgr', 'Distance Strata', 'sub id', 'sub name', 'rayon'}).reset_index(drop = True)

cig_fs = pd.merge(cig_fs, ops[['ccode', 'region', 'qct_name_NEW']].drop_duplicates(), on= 'ccode', how = 'left')
#cig_fs.to_excel('cig_free_search.xlsx')   

In [26]:
#beer 
beer = pt[pt['BEER'].notna() & pt['CIG'].isna() & pt['FMCG'].isna()].reset_index(drop = True)

beer_fs = pd.DataFrame(columns = ['Sample Area ID', 'cluster', 'ccode', 'cname', 'zadanie', 
                                  'pgr', 'Distance Strata', 'sub id', 'sub name', 'rayon'])

n = 0
for i, j in beer.iterrows():
    data = parser(rules['new_mcell_name'][rules['MCELL_ID'] == j['MICRO_CELL_RULE_ID']].iloc[0])
    if len(data[1]) == 1:
        beer_fs.loc[n, 'cname'] =  data[0][0]
        beer_fs.loc[n, 'ccode'] =  data[1][0]
        beer_fs.loc[n, 'zadanie'] = j['Sample Area ID']
        beer_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
        beer_fs.loc[n, 'cluster'] = j['cluster']
        n += 1
    elif len(data[1]) == 0:
        if j['rural'] == 1:
            cities = uo200[uo200['Federal Subject Number'].isin(data[2]) & 
                            uo200['pgr_MT'].isin(data[4]) & 
                                uo200['Distance Strata'].isin(data[8]) & 
                                    (uo200['RMS_CLUSTER_CODE (ACR)'].astype('str') == j['cluster'])]
            for k in cities.index:
                beer_fs.loc[n, 'cname'] =  cities['Town  / Village Name'][k]
                beer_fs.loc[n, 'ccode'] =  cities['ACN NN'][k]
                beer_fs.loc[n, 'zadanie'] = 'RURAL&' + str(int(j['Sample Area ID'])) 
                beer_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
                beer_fs.loc[n, 'cluster'] = j['cluster']
                beer_fs.loc[n, 'pgr'] = cities['pgr_MT'][k]
                beer_fs.loc[n, 'Distance Strata'] = cities['Distance Strata'][k]
                beer_fs.loc[n, 'sub id'] = cities['Federal Subject Number'][k]
                beer_fs.loc[n, 'sub name'] = cities['Federal Subject Name'][k]
                beer_fs.loc[n, 'rayon'] = cities['Rayon (for Villages only)'][k]
                n += 1
        else:
            cities = arc[['City Code', 'City Name (english)']][arc['# of Federal Subject'].isin(data[2]) & 
                                                               arc['pgr1'].isin(data[4]) & 
                                                               (arc['Restrict for audit '] == 0) & 
                                                               (arc['City Code'][~arc['City Code'].isin(data[6])]) & 
                                                        (arc['RMS_CLUSTER_CODE (справочник)'].astype('str') == j['cluster'])]
            for k in cities.index:
                beer_fs.loc[n, 'cname'] =  cities['City Name (english)'][k]
                beer_fs.loc[n, 'ccode'] =  cities['City Code'][k]
                beer_fs.loc[n, 'zadanie'] = 'OTHER&' + str(int(j['Sample Area ID'])) 
                beer_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
                beer_fs.loc[n, 'cluster'] = j['cluster']
                n += 1
                
rural_fs = pd.concat([rural_fs, beer_fs[beer_fs['ccode'].isna() | (beer_fs['ccode'] > 9999)]], ignore_index = True)
beer_fs = beer_fs[~beer_fs['ccode'].isna() & (beer_fs['ccode'] < 10000)].drop(columns = {'pgr', 'Distance Strata', 'sub id', 'sub name', 'rayon'}).reset_index(drop = True)

beer_fs = pd.merge(beer_fs, ops[['ccode', 'region', 'qct_name_NEW']].drop_duplicates(), on= 'ccode', how = 'left')
#beer_fs.to_excel('beer_free_search.xlsx')   
#rural_fs.to_excel('rural_free_search.xlsx')   

In [546]:
#exxon
exxon = pt[pt['EXXON'].notna()].reset_index(drop = True)

exxon_fs = pd.DataFrame(columns = ['Sample Area ID', 'cluster', 'ccode', 'cname', 'zadanie'])

n = 0
for i, j in exxon.iterrows():
    data = parser(rules['new_mcell_name'][rules['MCELL_ID'] == j['MICRO_CELL_RULE_ID']].iloc[0])
    if len(data[1]) == 1:
        exxon_fs.loc[n, 'cname'] =  data[0][0]
        exxon_fs.loc[n, 'ccode'] =  data[1][0]
        exxon_fs.loc[n, 'zadanie'] = j['Sample Area ID']
        exxon_fs.loc[n, 'Sample Area ID'] = j['Sample Area ID']
        exxon_fs.loc[n, 'cluster'] = j['cluster']
        n += 1

exxon_fs = pd.merge(exxon_fs, ops[['ccode', 'region', 'qct_name_NEW']].drop_duplicates(), on= 'ccode', how = 'left')

#exxon_fs.to_excel('exxon_free_search.xlsx')      

In [547]:
with pd.ExcelWriter('free_search.xlsx') as writer:
    free_search(df, mp).to_excel(writer, sheet_name='free_search', index = False)
    df[df['Stage'] == 'FRAME EXHAUST'].to_excel(writer, sheet_name='FRAME EXHAUST', index = False)
    fmcg_fs.to_excel(writer, sheet_name = 'fmcg', index = False)
    cig_fs.to_excel(writer, sheet_name = 'cig', index = False)
    beer_fs.to_excel(writer, sheet_name = 'beer', index = False)
    exxon_fs.to_excel(writer, sheet_name = 'exxon', index = False)
    rural_fs.to_excel(writer, sheet_name = 'rural', index = False)